# 03 - Carregamento (Loading)

## Objetivo

Este notebook implementa a etapa de **Carregamento (Loading)** do pipeline de dados. 

O processo de carregamento consiste em:
- Carregar os dados já transformados e refinados da camada **Silver (processed)**
- Aplicar validações finais e tratamentos de qualidade
- Salvar os dados no sistema de armazenamento final na camada **Gold**

A camada Gold representa o destino final dos dados, onde ficam disponíveis para consumo por analistas, cientistas de dados ou ferramentas de visualização.

---

## Ferramentas Utilizadas

- **pandas**: Manipulação de DataFrames
- **pyarrow**: Leitura e escrita de arquivos Parquet (formato otimizado para Big Data)
- **os**: Gerenciamento de sistema de arquivos

## 1. Importação de Bibliotecas


In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os


## 2. Configuração de Caminhos


In [4]:
# Caminhos das camadas do pipeline
PROCESSED_PATH = "../dados/processed/"
GOLD_PATH = "../dados/gold/"

# Criar diretório Gold se não existir
os.makedirs(GOLD_PATH, exist_ok=True)

print(f"📁 Caminho Silver (processed): {PROCESSED_PATH}")
print(f"📁 Caminho Gold (destino final): {GOLD_PATH}")


📁 Caminho Silver (processed): ../dados/processed/
📁 Caminho Gold (destino final): ../dados/gold/


## 3. Carregamento dos Dados da Camada Silver


In [5]:
# Carregar dados transformados da camada Silver
print("📥 Carregando dados da camada Silver...")
df = pd.read_csv(os.path.join(PROCESSED_PATH, "movies_full_cleaned.csv"))

print(f"✅ Dados carregados com sucesso!")
print(f"📊 Shape: {df.shape}")
print(f"📋 Colunas: {len(df.columns)}")


📥 Carregando dados da camada Silver...
✅ Dados carregados com sucesso!
📊 Shape: (8935, 11)
📋 Colunas: 11


## 4. Validações e Qualidade dos Dados

Antes de carregar os dados na camada Gold, realizamos validações finais para garantir a qualidade e integridade dos dados.


In [6]:
# Verificação de valores ausentes
print("=" * 60)
print("🔍 VALIDAÇÕES DE QUALIDADE")
print("=" * 60)

print(f"\n📊 Total de registros: {len(df):,}")
print(f"📋 Total de colunas: {len(df.columns)}")
print(f"\n🔢 Valores ausentes por coluna:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
    print(f"\n⚠️  Total de valores ausentes: {missing.sum()}")
    
    # Tratar valores ausentes - remover linhas com valores ausentes
    print("\n🧹 Removendo registros com valores ausentes...")
    df_clean = df.dropna().reset_index(drop=True)
    print(f"✅ Registros após limpeza: {len(df_clean):,}")
    print(f"📉 Registros removidos: {len(df) - len(df_clean)}")
    df = df_clean
else:
    print("✅ Nenhum valor ausente encontrado!")

print(f"\n📈 Dimensões finais: {df.shape}")
print(f"📝 Tipos de dados:\n{df.dtypes}")


🔍 VALIDAÇÕES DE QUALIDADE

📊 Total de registros: 8,935
📋 Total de colunas: 11

🔢 Valores ausentes por coluna:
id_genre      4
genre_name    4
dtype: int64

⚠️  Total de valores ausentes: 8

🧹 Removendo registros com valores ausentes...
✅ Registros após limpeza: 8,931
📉 Registros removidos: 4

📈 Dimensões finais: (8931, 11)
📝 Tipos de dados:
id                     int64
title                 object
runtime              float64
vote_average         float64
vote_count           float64
original_language     object
year                   int64
id_genre              object
genre_name            object
director              object
main_actor            object
dtype: object


## 5. Carregamento para Camada Gold

Os dados validados são carregados na camada Gold em dois formatos:
- **Parquet**: Formato otimizado, com compressão e preservação de tipos
- **CSV**: Formato legível e compatível para backup/referência


In [7]:
print("=" * 60)
print("💾 CARREGANDO DADOS NA CAMADA GOLD")
print("=" * 60)

# Salvar em formato Parquet (otimizado para Big Data)
parquet_path = os.path.join(GOLD_PATH, "movies_final.parquet")
print(f"\n📦 Salvando em Parquet: {parquet_path}")
df.to_parquet(parquet_path, index=False)
print("✅ Arquivo Parquet salvo com sucesso!")

# Verificar tamanho do arquivo Parquet
parquet_size = os.path.getsize(parquet_path) / (1024 * 1024)  # MB
print(f"📏 Tamanho do arquivo Parquet: {parquet_size:.2f} MB")


💾 CARREGANDO DADOS NA CAMADA GOLD

📦 Salvando em Parquet: ../dados/gold/movies_final.parquet
✅ Arquivo Parquet salvo com sucesso!
📏 Tamanho do arquivo Parquet: 0.39 MB


In [8]:
# Salvar em formato CSV (backup e compatibilidade)
csv_path = os.path.join(GOLD_PATH, "movies_final.csv")
print(f"\n📄 Salvando em CSV: {csv_path}")
df.to_csv(csv_path, index=False)
print("✅ Arquivo CSV salvo com sucesso!")

# Verificar tamanho do arquivo CSV
csv_size = os.path.getsize(csv_path) / (1024 * 1024)  # MB
print(f"📏 Tamanho do arquivo CSV: {csv_size:.2f} MB")
print(f"💡 Compressão Parquet: {((1 - parquet_size/csv_size) * 100):.1f}% menor que CSV")



📄 Salvando em CSV: ../dados/gold/movies_final.csv
✅ Arquivo CSV salvo com sucesso!
📏 Tamanho do arquivo CSV: 0.93 MB
💡 Compressão Parquet: 57.6% menor que CSV


## 6. Resumo Final


In [9]:
print("=" * 60)
print("✅ CARREGAMENTO CONCLUÍDO COM SUCESSO!")
print("=" * 60)
print(f"\n📊 Resumo dos dados carregados na camada Gold:")
print(f"   • Total de registros: {len(df):,}")
print(f"   • Total de colunas: {len(df.columns)}")
print(f"   • Arquivo Parquet: {parquet_path}")
print(f"   • Arquivo CSV: {csv_path}")
print(f"\n🎯 Os dados estão prontos para consumo!")
print(f"   - Análises exploratórias")
print(f"   - Visualizações")
print(f"   - Modelagem de machine learning")
print("=" * 60)


✅ CARREGAMENTO CONCLUÍDO COM SUCESSO!

📊 Resumo dos dados carregados na camada Gold:
   • Total de registros: 8,931
   • Total de colunas: 11
   • Arquivo Parquet: ../dados/gold/movies_final.parquet
   • Arquivo CSV: ../dados/gold/movies_final.csv

🎯 Os dados estão prontos para consumo!
   - Análises exploratórias
   - Visualizações
   - Modelagem de machine learning
